# 1-1 视觉生成方法

 (目录)

## 0 章节目标
- 了解视觉生成概念
- 理解视觉生成的主要方法，并且能运用这些方法去解决问题

## 1 概念
在AIGC的时代，除了大家熟知的自然语言生成大模型非常重要以外，像DALL-E2、Stable-Diffusion、Midjourney等视觉生成大模型也同样重要，并且其视觉生成大模型在绘画、影视、自动驾驶、机器人、商业、游戏等诸多领域有着非常重要的应用。
### 1.1 什么是视觉生成大模型？
### 1.2 视觉生成模型与自然语言生成模型的区别？

## 2 方法

### 2.1 生成对抗网络（GAN）

### 2.2 变分自编码网络（VAE）

### 2.3 流式模型（flow-based model）

### 2.4 扩散模型（Diffusion model）